In [8]:
from autogen import AssistantAgent, UserProxyAgent
from constants import model_name, base_url, api_key
import re
import json

# Helper function to save the JSON-like string to a file
def save_json_like_string(json_like_str, filename):
    # Remove TERMINATE and strip whitespace
    cleaned = re.sub(r'TERMINATE\s*$', '', json_like_str.strip())
    # Remove unnecessary newlines (but keep inside strings)
    cleaned = cleaned.replace('\n', '')
    # Parse the string to a Python dict
    try:
        data = json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        return
    # Save to file
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

# Configuration for the language model
print("Using model:", model_name)
llm_config = {"config_list": [{
            "model": model_name,  # or other supported model IDs
            "base_url": base_url,
            "api_key": api_key,
        }]}


# Define the system prompt
system_prompt_cosmetics = """You will get an idea from the user regarding the cosmetics they want to create. Analyze the idea and create a detailed formula for a cosmetic product that meets the user's objectives. The formula should include the following sections:
1. Product Name: A catchy and relevant name for the cosmetic product.
2. Description: A brief description of the product, its benefits, and its target audience.
3. Ingredients: A detailed list of ingredients, including their functions and concentrations.
4. Instructions for Use: Clear and concise instructions on how to use the product.
5. Packaging: Suggestions for packaging that aligns with the product's branding and target audience.
6. Safety and Regulatory Information: Any necessary safety warnings or regulatory information.
7. Additional Notes: Any other relevant information or tips for the user.
Return the formula in a structured format, such as JSON or a well-organized text format. 
After the formula is created, return "TERMINATE" to end the conversation.
"""

# Create the formula creator agent with the defined system prompt
formula_creator = AssistantAgent(
    name="Formula_Creator",
    system_message=system_prompt_cosmetics,
    llm_config=llm_config,
)

# Define the user proxy agent to simulate user input and start the chat
user_proxy_cosmetics = UserProxyAgent(
        name="User",
        human_input_mode="NEVER", # Set to "ALWAYS" to allow and ask for human input
        max_consecutive_auto_reply=10,
        is_termination_msg=lambda x: x.get("content", "").strip().endswith("TERMINATE"),
        code_execution_config={"work_dir": "coding", "use_docker": False},
    )
    


Using model: deepseek/deepseek-chat-v3.1:free


In [5]:
message_cosmetics = input('Enter your objectives for the cosmetics : ')
chat_result = user_proxy_cosmetics.initiate_chat(
formula_creator,
message=message_cosmetics
)

User (to Formula_Creator):

I want a premium shampoo for men. Should smell good, be of a good color, and clean well (in particular, suitable for oily hair)

--------------------------------------------------------------------------------
[autogen.oai.client: 09-22 09:42:16] {652} WARNING - Model deepseek/deepseek-chat-v3.1:free is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Formula_Creator (to User):

{
  "ProductName": "Apex Cleanse - Men's Premium Clarifying Shampoo",
  "Description": "Apex Cleanse is a premium clarifying shampoo designed specifically for men with oily hair. It deeply cleanses the scalp and hair, removing excess oil and impurities without stripping natural moisture. Infused with a sophisticated masculine scent of cedarwood and bergamot, it leaves hair feeling fresh, light, and revitalized. The rich, deep blue color enhances the premium experience. Ideal for acti

In [6]:
formula = chat_result.chat_history[-1]['content']
formula

'{\n  "ProductName": "Apex Cleanse - Men\'s Premium Clarifying Shampoo",\n  "Description": "Apex Cleanse is a premium clarifying shampoo designed specifically for men with oily hair. It deeply cleanses the scalp and hair, removing excess oil and impurities without stripping natural moisture. Infused with a sophisticated masculine scent of cedarwood and bergamot, it leaves hair feeling fresh, light, and revitalized. The rich, deep blue color enhances the premium experience. Ideal for active men seeking a reliable daily cleanser that combats oiliness while providing a luxurious feel.",\n  "Ingredients": [\n    {"Ingredient": "Aqua", "Function": "Solvent base", "Concentration": "To 100%"},\n    {"Ingredient": "Sodium Lauryl Sulfoacetate", "Function": "Gentle surfactant", "Concentration": "12%"},\n    {"Ingredient": "Cocamidopropyl Betaine", "Function": "Foam booster, mild cleanser", "Concentration": "5%"},\n    {"Ingredient": "Glycerin", "Function": "Humectant, moisture retention", "Conce

In [9]:
save_json_like_string(formula, 'formula.json')